In [1]:
import pandas as pd
import numpy as np

In [4]:
file1=pd.read_csv('D:/Users/Kate/onedrive/git/mental-health/data.csv', sep='\t')
file1=file1.dropna()

In [6]:
file1=file1.dropna()

In [7]:
file1.head()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,screensize,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major
3,2,23,5081,3,11,6837,2,37,5521,1,...,2,1,2,4,5,70,2,1,5,biology
4,2,36,3215,2,13,7731,3,5,4156,4,...,2,2,3,10,1,10,2,1,4,Psychology
6,1,20,4325,1,34,4009,2,38,3604,3,...,2,1,1,7,2,60,2,1,4,Mechatronics engeenerieng
7,1,34,4796,1,9,2618,1,39,5823,1,...,2,1,1,2,2,60,1,1,2,Music
8,4,4,3470,4,14,2139,3,1,11043,4,...,1,1,1,12,2,70,2,1,4,Psychology


In [14]:
columns_names=file1.columns
scale_q_names=columns_names[:126]
scale_q_data=file1[scale_q_names]
scale_q_data.describe()

,Q1A,Q1I,Q1E,Q2A,Q2I,Q2E,Q3A,Q3I,Q3E,Q4A,...,Q39E,Q40A,Q40I,Q40E,Q41A,Q41I,Q41E,Q42A,Q42I,Q42E
count,28371.000000,28371.000000,2.837100e+04,28371.00000,28371.000000,2.837100e+04,28371.000000,28371.000000,2.837100e+04,28371.000000,...,2.837100e+04,28371.000000,28371.000000,2.837100e+04,28371.000000,28371.000000,2.837100e+04,28371.000000,28371.000000,2.837100e+04
mean,2.571605,21.503648,6.924198e+03,2.15752,21.262275,5.608689e+03,2.154207,21.599450,6.639495e+03,1.881922,...,9.463701e+03,2.584999,21.554968,8.429540e+03,1.890240,21.462479,6.007679e+03,2.584188,21.470727,8.168577e+03
std,1.025603,12.144281,6.612065e+04,1.10673,12.131935,2.787799e+04,1.019139,12.152251,3.625873e+04,1.013244,...,1.990002e+05,1.101986,12.127491,1.033421e+05,1.010488,12.101042,7.038782e+04,1.023820,12.102148,5.152566e+04
min,1.000000,1.000000,1.800000e+02,1.00000,1.000000,1.810000e+02,1.000000,1.000000,1.780000e+02,1.000000,...,1.780000e+02,1.000000,1.000000,1.780000e+02,1.000000,1.000000,-1.590000e+02,1.000000,1.000000,1.780000e+02
25%,2.000000,11.000000,2.736000e+03,1.00000,11.000000,2.579000e+03,1.000000,11.000000,2.951000e+03,1.000000,...,2.189000e+03,2.000000,11.000000,3.505000e+03,1.000000,11.000000,2.304000e+03,2.000000,11.000000,3.211000e+03
50%,2.000000,22.000000,3.706000e+03,2.00000,21.000000,3.662000e+03,2.000000,22.000000,4.029000e+03,2.000000,...,3.045000e+03,3.000000,22.000000,4.777000e+03,2.000000,21.000000,3.141000e+03,2.000000,21.000000,4.575000e+03
75%,3.000000,32.000000,5.505000e+03,3.00000,32.000000,5.452000e+03,3.000000,32.000000,5.974500e+03,2.000000,...,5.366500e+03,4.000000,32.000000,6.954000e+03,2.000000,32.000000,4.654000e+03,3.000000,32.000000,7.016500e+03
max,4.000000,42.000000,8.549589e+06,4.00000,42.000000,2.161057e+06,4.000000,42.000000,4.339463e+06,4.000000,...,3.177322e+07,4.000000,42.000000,1.601340e+07,4.000000,42.000000,8.021110e+06,4.000000,42.000000,4.385661e+06


In [16]:
columns_names[126:]

Index(['country', 'source', 'introelapse', 'testelapse', 'surveyelapse',
       'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4', 'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8',
       'TIPI9', 'TIPI10', 'VCL1', 'VCL2', 'VCL3', 'VCL4', 'VCL5', 'VCL6',
       'VCL7', 'VCL8', 'VCL9', 'VCL10', 'VCL11', 'VCL12', 'VCL13', 'VCL14',
       'VCL15', 'VCL16', 'education', 'urban', 'gender', 'engnat', 'age',
       'screensize', 'uniquenetworklocation', 'hand', 'religion',
       'orientation', 'race', 'voted', 'married', 'familysize', 'major'],
      dtype='object')

In [21]:
tech_names=columns_names[126:131]
data_tech=file1[tech_names]

scale_t_names=columns_names[131:141]
scale_t_data=file1[scale_t_names]

scale_v_names=columns_names[141:157]
scale_v_data=file1[scale_t_names]

personal_info_names=columns_names[157:]
persocal_info=file1[personal_info_names]

In [22]:
persocal_info.describe()

,education,urban,gender,engnat,age,screensize,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize
count,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000,28371.000000
mean,2.792323,2.217158,1.799619,1.718868,24.619012,1.247647,1.227874,1.125903,8.208417,1.619682,26.677241,1.659194,1.167389,3.728913
std,0.790639,0.802074,0.421398,0.451750,24.763635,0.431653,0.419468,0.381372,3.179035,1.358901,24.829531,0.488347,0.444132,2.203292
min,0.000000,0.000000,0.000000,0.000000,13.000000,1.000000,1.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,2.000000,1.000000,20.000000,1.000000,1.000000,1.000000,7.000000,1.000000,10.000000,1.000000,1.000000,2.000000
50%,3.000000,2.000000,2.000000,2.000000,22.000000,1.000000,1.000000,1.000000,10.000000,1.000000,10.000000,2.000000,1.000000,3.000000
75%,3.000000,3.000000,2.000000,2.000000,26.000000,1.000000,1.000000,1.000000,10.000000,2.000000,60.000000,2.000000,1.000000,5.000000
max,4.000000,3.000000,3.000000,2.000000,1998.000000,2.000000,2.000000,3.000000,12.000000,5.000000,70.000000,2.000000,3.000000,133.000000


In [24]:
file1['Q1E'].describe()

count    2.837100e+04
mean     6.924198e+03
std      6.612065e+04
min      1.800000e+02
25%      2.736000e+03
50%      3.706000e+03
75%      5.505000e+03
max      8.549589e+06
Name: Q1E, dtype: float64